In [1]:
from fredapi import Fred
from dotenv import load_dotenv
import os
import json
import pandas as pd

load_dotenv()

# FRED_API_KEY = os.getenv("FRED_API_KEY")

# fred = Fred(api_key=FRED_API_KEY)

True

In [ ]:
def pull_fred_series(series_id: str, series_name: str):
    """
    Fetches a time series from FRED by its series ID and converts it to a pandas Series dataframe.
    """
    try:
        fred = Fred(api_key=os.getenv("FRED_API_KEY"))
        series = fred.get_series(series_id)
        df = series.to_frame().reset_index()
        df.columns = ['Date', series_name]
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

        return df
    
    except Exception as e:
        print(f"Error fetching series {series_id}: {e}")
        
        return None


def series_to_json(series_id, category, df):
    """
    Convert a FRED series to a JSON string, saving into a category subfolder.
    Creates the folder if it does not exist.
    """
    folder = f"data/{category}"
    os.makedirs(folder, exist_ok=True)  # <-- ensure subfolder exists

    path = f"{folder}/fred_{series_id}.json"

    if os.path.exists(path):
        with open(path, "r") as f:
            existing_data = json.load(f)
        df_existing = pd.DataFrame(existing_data)
        
        df_existing["Date"] = pd.to_datetime(df_existing["Date"])

        df_combined = pd.concat([df_existing, df])
        df_combined = (
            df_combined.drop_duplicates(subset=["Date"])
            .sort_values("Date")
            .reset_index(drop=True)
        )
        df_combined["Date"] = df_combined["Date"].dt.strftime("%Y-%m-%d")
    else:
        df_combined = df

    df_combined.to_json(path, orient="records")
    
    return True


def fetch_fred_series(category, series_id, start_date=None, end_date=None):
    df = pd.read_json(f"data/{category}/fred_{series_id}.json")
    df["Date"] = pd.to_datetime(df["Date"])
    
    if start_date:
        df = df[df["Date"] >= start_date]
    if end_date:
        df = df[df["Date"] <= end_date]
    
    return df


def load_registry(path: str):
    with open(path, "r") as f:
        return json.load(f)


def refresh_from_registry(path="data/registry.json"):
    registry = load_registry(path)
    for entry in registry:
        sid = entry["id"]
        name = entry["name"]
        category = entry["category"]
        # pull from FRED
        df = pull_fred_series(sid, name)
        if df is not None:
            series_to_json(sid, category, df)

In [2]:
def pull_fred_series(series_id: str, series_name: str):
    """
    Fetch a time series from FRED by its series ID and convert it to a pandas DataFrame.
    Returns DataFrame with Date as datetime64.
    """
    try:
        fred = Fred(api_key=os.getenv("FRED_API_KEY"))
        series = fred.get_series(series_id)
        df = series.to_frame().reset_index()
        df.columns = ["Date", series_name]
        # keep Date as datetime64 here
        return df
    except Exception as e:
        print(f"Error fetching series {series_id}: {e}")
        return None


def series_to_json(series_id, category, df):
    """
    Merge new data with existing JSON (if any) and save back to disk.
    JSON always stores Date as 'YYYY-MM-DD' strings.
    """
    folder = f"data/{category}"
    os.makedirs(folder, exist_ok=True)

    path = f"{folder}/fred_{series_id}.json"

    if os.path.exists(path):
        with open(path, "r") as f:
            existing_data = json.load(f)
        df_existing = pd.DataFrame(existing_data)
        df_existing["Date"] = pd.to_datetime(df_existing["Date"])
        df_combined = pd.concat([df_existing, df])
    else:
        df_combined = df

    df_combined = (
        df_combined.drop_duplicates(subset=["Date"])
        .sort_values("Date")
        .reset_index(drop=True)
    )

    # save as string for JSON
    df_combined["Date"] = df_combined["Date"].dt.strftime("%Y-%m-%d")

    df_combined.to_json(path, orient="records")
    print(f"Updated data saved to {path}")

    return True


def fetch_fred_series(category, series_id, start_date=None, end_date=None):
    """
    Load series from local JSON and optionally filter by start_date/end_date.
    Returns DataFrame with Date as datetime64.
    """
    path = f"data/{category}/fred_{series_id}.json"
    df = pd.read_json(path)

    # convert to datetime on load
    df["Date"] = pd.to_datetime(df["Date"])

    if start_date:
        df = df[df["Date"] >= pd.to_datetime(start_date)]
    if end_date:
        df = df[df["Date"] <= pd.to_datetime(end_date)]

    return df


def load_registry(path: str):
    """
    Load the series registry from JSON.
    """
    with open(path, "r") as f:
        return json.load(f)


def refresh_from_registry(path="data/registry.json"):
    """
    Pull and refresh all series defined in the registry JSON.
    """
    registry = load_registry(path)
    for entry in registry:
        sid = entry["id"]
        name = entry["name"]
        category = entry["category"]
        df = pull_fred_series(sid, name)
        if df is not None:
            series_to_json(sid, category, df)


In [5]:
refresh_from_registry()

Updated data saved to data/housing/fred_MSPUS.json
Updated data saved to data/housing/fred_MSPNHSUS.json
Updated data saved to data/housing/fred_CSUSHPINSA.json
Updated data saved to data/housing/fred_NHFSEPNTS.json
Updated data saved to data/housing/fred_NHFSEPUCS.json
Updated data saved to data/housing/fred_NHFSEPCS.json
Updated data saved to data/housing/fred_HNFSUSNSA.json
Updated data saved to data/commodities/fred_APU0000708111.json
Updated data saved to data/commodities/fred_APU0000709112.json
Updated data saved to data/commodities/fred_APU0000703112.json
Updated data saved to data/commodities/fred_APU0000702111.json
Updated data saved to data/commodities/fred_APU0000FF1101.json
Updated data saved to data/commodities/fred_APU000074714.json
Updated data saved to data/commodities/fred_APU000072610.json
Updated data saved to data/commodities/fred_APU0000717311.json
Updated data saved to data/commodities/fred_APU0000704111.json
Updated data saved to data/delinquency/fred_DRCCLACBS.j

In [ ]:
df = pull_fred_series(series_id="MSPUS", series_name="Median Home Sales Price")


In [ ]:
series_to_json(series_id="MSPUS", category="housing", df=df)

In [ ]:
json_df = fetch_fred_series(category="housing", series_id="MSPUS")
json_df.head()